In [1]:
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/che_tools.py
!pip install DotMap

--2020-09-07 13:21:35--  https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5583 (5.5K) [text/plain]
Saving to: ‘trust_ncg.py’

trust_ncg.py        100%[===================>]   5.45K  --.-KB/s    in 0s      

2020-09-07 13:21:35 (47.9 MB/s) - ‘trust_ncg.py’ saved [5583/5583]

--2020-09-07 13:21:35--  https://raw.githubusercontent.com/profteachkids/CHE2064/master/che_tools.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6401 (6.3K) [text/plain]
Saving to: ‘che_tools.

In [2]:
import trust_ncg as tncg
import jax
import jax.numpy as jnp
import copy
import che_tools as che
from dotmap import DotMap
import pandas as pd

p = che.Props(['Ethanol', 'Isopropanol', 'Water'])

In [3]:
s=DotMap()
s.F = jnp.array([1/3, 1/3, 1/3])
s.FT = 350
s.flashP= 101325
s.q = 0.5 #fraction liquid feed

v=DotMap()
v.Vq = che.xtoq([0.4, 0.35, 0.25])
v.Lq = che.xtoq([0.25, 0.35, 0.4])
v.flashT = 300
v.Vtot = jnp.sum(s.F)/2.
v.Ltot = jnp.sum(s.F)/2.

/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


[Traced<ShapedArray(float64[], weak_type=True):JaxprTrace(level=-1/1)>, Traced<ShapedArray(float64[], weak_type=True):JaxprTrace(level=-1/1)>, Traced<ShapedArray(float64[], weak_type=True):JaxprTrace(level=-1/1)>]


In [4]:
def model(c):
    c=DotMap(c)
    x=che.qtox(c.Vq)
    y=che.qtox(c.Lq)
    V = x * c.Vtot
    L = y * c.Ltot
    mass_balance = (c.F - V - L)
    mass_balance = (mass_balance/jnp.sum(c.F))**2

    FH = p.Hmix( (1.-c.q)*c.F, c.q*c.F, c.FT)
    VH = p.Hmix(V, 0, c.flashT)
    LH = p.Hmix(0, L, c.flashT)
    energy_balance = (FH - VH - LH)
    energy_balance = (energy_balance/jnp.sum(FH))**2

    fugacity = (x*p.Pvap(c.flashT) - y*c.flashP)
    fugacity = (fugacity/jnp.sum(y*c.flashP))**2
    # print(mass_balance.)
    # print(energy_balance)
    # print(fugacity)

    return jnp.sum(mass_balance + energy_balance + fugacity)

In [5]:
model_f, guess_f , transform_sol= tncg.transform(model,v,s)
model_f = jax.jit(model_f)

In [6]:
guess_f

DeviceArray([-4.70003629e-01, -1.33531393e-01,  5.00000000e-01,
              4.70003629e-01,  3.36472237e-01,  5.00000000e-01,
              3.00000000e+02], dtype=float64)

In [7]:
x_min, model_min = tncg.minimize(model_f, guess_f, verbose=True)
x_min, df_x_min, combined, df_combined, x_tree= transform_sol(x_min)

0:5, f: 0.3182956617493208
x: [ 1.44269061e-01 -6.53666781e-02  3.66904750e-01 -1.26643113e-01
  3.58293048e-02  6.19800357e-01  3.00373608e+02]
grad: [-0.04721111 -0.00737482 -0.23497796  0.01166882 -0.01378311 -0.47082382
 -0.002646  ]
dx: [ 0.61427269  0.06816471 -0.13309525 -0.59664674 -0.30064293  0.11980036
  0.37360762]

1:6, f: 0.29481745844341756
x: [5.62298623e-02 4.44164391e-02 3.96339396e-01 2.31489098e-01
 2.20352242e-01 6.01420712e-01 3.02327241e+02]
grad: [ 0.02232552 -0.02059477  0.00349715 -0.00930881  0.00272291  0.01913257
 -0.00229122]
dx: [-0.0880392   0.10978312  0.02943465  0.35813221  0.18452294 -0.01837965
  1.95363384]

2:5, f: 0.28723735962710917
x: [ 7.68557878e-02  1.24923415e-01  2.78646174e-01  1.74647430e-01
 -2.57054697e-01  6.75721962e-01  3.06294933e+02]
grad: [ 0.00264648  0.00452797  0.00026278  0.00167349  0.00466356  0.00464323
 -0.0026232 ]
dx: [ 0.02062593  0.08050698 -0.11769322 -0.05684167 -0.47740694  0.07430125
  3.96769108]

3:5, f: 0.28070

In [8]:
DotMap(x_min).pprint()

{'Lq': DeviceArray([0.40101171, 0.33641055], dtype=float64),
 'Ltot': DeviceArray([0.52861241], dtype=float64),
 'Vq': DeviceArray([-0.4140908 , -0.32138035], dtype=float64),
 'Vtot': DeviceArray([0.47138759], dtype=float64),
 'flashT': DeviceArray([359.96648377], dtype=float64)}


In [9]:
df_x_min


v2                  v1
              x1        x2        x1
Lq      0.401012  0.336411          
Ltot                        0.528612
Vq     -0.414091  -0.32138          
Vtot                        0.471388
flashT                       359.966

In [10]:
DotMap(combined).pprint()

{'F': DeviceArray([0.33333333, 0.33333333, 0.33333333], dtype=float64),
 'FT': DeviceArray([350.], dtype=float64),
 'Lq': DeviceArray([0.40101171, 0.33641055], dtype=float64),
 'Ltot': DeviceArray([0.52861241], dtype=float64),
 'Vq': DeviceArray([-0.4140908 , -0.32138035], dtype=float64),
 'Vtot': DeviceArray([0.47138759], dtype=float64),
 'flashP': DeviceArray([101325.], dtype=float64),
 'flashT': DeviceArray([359.96648377], dtype=float64),
 'q': DeviceArray([0.5], dtype=float64)}


In [ ]:
df_combined

In [ ]:
x_tree
idx = pd.IndexSlice
df2=df_combined.loc[idx[:,'q'],('v3')]
df3 = df2.style.format('{:6.2f}' )
df3